# Introduction to Memory Profiling

> Objectives:
> * Be introduced to memory profiling using different tools
> * Some small introduction to time profiling in IPython too


## ipython_memwatcher

Our recommended way to profile memory consumption for this tutorial will be [ipython_memwatcher](https://pypi.python.org/pypi/ipython_memwatcher):


In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.027 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 34.930 MiB


In [2]:
# Let's create a big object
a = [i for i in range(1000*1000)]

In [2] used 31.121 MiB RAM in 0.275s, peaked 4.094 MiB above current, total RAM usage 66.051 MiB


In [3]:
# Get some measurements from the last executed cell:
meas = mw.measurements
meas

Measurements(memory_delta=31.12109375, time_delta=0.2749760150909424, memory_peak=4.09375, memory_usage=66.05078125)

In [3] used 0.035 MiB RAM in 0.048s, peaked 0.000 MiB above current, total RAM usage 66.086 MiB


In [4]:
# MemWatcher.measurements is a named tuple.  We can easily get info out of it:
meas.memory_delta

31.12109375

In [4] used 0.004 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 66.090 MiB


In [5]:
# This takes between 32 ~ 35 bytes per element:
meas.memory_delta * (2**20) / 1e6

32.632832

In [5] used 0.016 MiB RAM in 0.010s, peaked 0.000 MiB above current, total RAM usage 66.105 MiB


In [6]:
# What are these elements made from?
type(a[0])

int

In [6] used 0.000 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 66.105 MiB


In [7]:
# How much memory take an int?
# Beware: the size below will depend on whether you are using a 32-bit or 64-bit Python
import sys
sys.getsizeof(a[0])

24

In [7] used 0.008 MiB RAM in 0.012s, peaked 0.000 MiB above current, total RAM usage 66.113 MiB


Ok.  On 64-bits, that means that the int object allocates 8 bytes for the integer value, and 16 bytes for other metadata (Python object).  But 24 is quite less than 32~35.  Where this overhead comes from?

Well, it turns out that the list structure needs additional pointers to reference the different components.

![List diagram](http://www.brpreiss.com/books/opus7/html/img579.gif)

## memory_profiler

[memory_profiler](https://pypi.python.org/pypi/memory_profiler) is a basic module for memory profiling that many others use (like the `ipython_memwatcher` above) and it interacts well with ipython, so it is worth to see how it works:

In [8]:
%load_ext memory_profiler

In [8] used 0.070 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 66.184 MiB


In [9]:
# Use %memit magic command exposed by memory_profiler
%memit b = [i for i in range(1000*1000)]

peak memory: 105.27 MiB, increment: 39.07 MiB
In [9] used 39.309 MiB RAM in 0.523s, peaked 0.000 MiB above current, total RAM usage 105.492 MiB


Please note that the `peak_memory` in this case is different than the `peaked_memory` reported by ipython_memwatcher package.

## Guppy

Guppy is nice for having an overview of how different structures are using our memory:

In [10]:
from guppy import hpy; hp=hpy()
hp.heap()

Partition of a set of 2146512 objects. Total size = 84880200 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 2003288  93 48078912  57  48078912  57 int
     1   1816   0 16501632  19  64580544  76 list
     2  69505   3  6527248   8  71107792  84 str
     3  33715   2  2888640   3  73996432  87 tuple
     4   1814   0  1797392   2  75793824  89 dict (no owner)
     5    480   0  1405440   2  77199264  91 dict of module
     6   8649   0  1107072   1  78306336  92 types.CodeType
     7   8424   0  1010880   1  79317216  93 function
     8   1021   0   920176   1  80237392  95 type
     9   1021   0   835000   1  81072392  96 dict of type
<657 more rows. Type e.g. '_.more' to view.>

In [10] used 68.152 MiB RAM in 13.666s, peaked 0.000 MiB above current, total RAM usage 173.645 MiB


In [11]:
del b
hp.heap()

Partition of a set of 1146787 objects. Total size = 52765960 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 1003546  88 24085104  46  24085104  46 int
     1   1813   0  8374952  16  32460056  62 list
     2  69510   6  6527568  12  38987624  74 str
     3  33716   3  2888744   5  41876368  79 tuple
     4   1820   0  1800608   3  43676976  83 dict (no owner)
     5    480   0  1405440   3  45082416  85 dict of module
     6   8649   1  1107072   2  46189488  88 types.CodeType
     7   8423   1  1010760   2  47200248  89 function
     8   1021   0   920176   2  48120424  91 type
     9   1021   0   835000   2  48955424  93 dict of type
<653 more rows. Type e.g. '_.more' to view.>

In [11] used 12.312 MiB RAM in 2.690s, peaked 0.000 MiB above current, total RAM usage 185.957 MiB


In [12]:
# Size of the list (beware, this does not include the contents!)
hp.iso(a)

Partition of a set of 1 object. Total size = 8126536 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1 100  8126536 100   8126536 100 list

In [12] used 0.016 MiB RAM in 0.014s, peaked 0.000 MiB above current, total RAM usage 185.973 MiB


## %time and %timeit

In [13]:
# IPython provides a magic command to see how much time a command takes to run
%time asum = sum(a)

CPU times: user 28 ms, sys: 4 ms, total: 32 ms
Wall time: 30.5 ms
In [13] used 0.012 MiB RAM in 0.037s, peaked 0.000 MiB above current, total RAM usage 185.984 MiB


Note that `%time` offers quite detailed statistics on the time spent.

Also, the time reported by MemoryWatcher has a typical overhead of 1~5 ms over the time reported by %time, so when the times to measure are about this order then it is better to rely on the %time (or %timeit below) values.  

In [14]:
# We have another way to measure timings doing several loops and getting the mean
%timeit bsum = sum(a)
# However, one must notice that %timeit does not return the result of expressions

10 loops, best of 3: 22 ms per loop
In [14] used 0.023 MiB RAM in 0.946s, peaked 0.000 MiB above current, total RAM usage 186.008 MiB


Interestingly, %timeit allows to retrieve the measured times in loops with the -o flag:

In [15]:
t = %timeit -o sum(a)
print(t.all_runs)
print(t.best)

10 loops, best of 3: 22.1 ms per loop
[0.22296881675720215, 0.2211289405822754, 0.22654008865356445]
0.0221128940582
In [15] used 0.016 MiB RAM in 1.052s, peaked 0.000 MiB above current, total RAM usage 186.023 MiB


And one can specify the number of loops (-n) and the number of repetitions (-r):

In [16]:
t = %timeit -r1 -n1 -o sum(a)
print(t.all_runs)
print(t.best)

1 loop, best of 1: 29.6 ms per loop
[0.029569149017333984]
0.0295691490173
In [16] used 0.008 MiB RAM in 0.049s, peaked 0.000 MiB above current, total RAM usage 186.031 MiB


### Exercise 1

Provided a dictionary like:

```
d = dict(("key: %i"%i, i*2) for i in a)
```

Try to guess how much RAM it uses using the techniques introduced above.

Why do you think it takes more space than a list?

*Hint*: Every entry in a dictionary has pointers to two objects: key and value. 

## Solution

In [20]:
d = dict(("key: %i"%i, i*2) for i in a)

In [20] used 103.508 MiB RAM in 2.975s, peaked 0.000 MiB above current, total RAM usage 399.398 MiB


In [18]:
# Compute the size of key + value
sys.getsizeof("key: 100000") + sys.getsizeof(1)

72

In [18] used 0.008 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 295.887 MiB


In [19]:
# Compute the size of the dict struct (using guppy here)
hp.iso(d)

Partition of a set of 1 object. Total size = 50331928 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1 100 50331928 100  50331928 100 dict (no owner)

In [19] used 0.004 MiB RAM in 2.018s, peaked 0.000 MiB above current, total RAM usage 295.891 MiB


So, guppy is telling us that just the dictionary structure is taking ~50 MB, whereas the contents alone are taking ~70MB, so we should have expected the dictionary to consume ~120 MB.  However, our `MemWatcher` instance is reporting just ~100 MB.  Take away lesson: measuring memory consumption in Python is tricky but using proper tools we can still get valuable hints!